In [1]:
!pip install tensorflow keras torch torchvision opencv-python matplotlib pandas jsonlines scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import tensorflow as tf
import json
import requests
import numpy as np
from tensorflow.keras.applications import (
    EfficientNetB0, MobileNetV2, ResNet50, VGG16, VGG19, DenseNet121, DenseNet201,
    Xception, InceptionV3, NASNetMobile
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from io import BytesIO
from PIL import Image

# Function to download and process images
def load_data(jsonl_path):
    images, labels = [], []
    with open(jsonl_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            try:
                image_url = data['messages'][2]['content'][0]['image_url']['url']
                label = data['messages'][3]['content']

                # Download image
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content)).convert('RGB')
                img = img.resize((224, 224))  # Resize for model input
                img_array = image.img_to_array(img)
                img_array = preprocess_input(img_array)

                images.append(img_array)
                labels.append(label)
            except KeyError as e:
                print(f"Skipping entry due to missing key: {e}")

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Load train, validation, and test data
train_images, train_labels = load_data("train.jsonl")
valid_images, valid_labels = load_data("valid.jsonl")
test_images, test_labels = load_data("test.jsonl")

# Label encoding
label_mapping = {'Cleanliness Issues': 0, 'Infrastructure Problem': 1, 'Overcrowding': 2, 'Suspicious': 3}
train_labels = np.array([label_mapping[label] for label in train_labels])
valid_labels = np.array([label_mapping[label] for label in valid_labels])
test_labels = np.array([label_mapping[label] for label in test_labels])

# Convert to categorical
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=4)
valid_labels = tf.keras.utils.to_categorical(valid_labels, num_classes=4)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=4)

# Function to create model
def create_model(base_model):
    base_model.trainable = False  # Freeze base model
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Define 10 models
models = {
    "EfficientNetB0": create_model(EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "MobileNetV2": create_model(MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "ResNet50": create_model(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "VGG16": create_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "VGG19": create_model(VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "DenseNet121": create_model(DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "DenseNet201": create_model(DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "Xception": create_model(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "InceptionV3": create_model(InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "NASNetMobile": create_model(NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
}

# Compile models
for model_name, model in models.items():
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train models
history = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(train_images, train_labels, epochs=5, validation_data=(valid_images, valid_labels))
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    history[model_name] = test_acc

# Print test accuracy for each model
for model_name, acc in history.items():
    print(f"{model_name} Test Accuracy: {acc:.2f}")


Training EfficientNetB0...
Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 6s/step - accuracy: 0.4499 - loss: 1.2352 - val_accuracy: 0.8000 - val_loss: 0.7023
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.8599 - loss: 0.5861 - val_accuracy: 0.9333 - val_loss: 0.4488
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step - accuracy: 0.9874 - loss: 0.2931 - val_accuracy: 0.9333 - val_loss: 0.3261
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 1.0000 - loss: 0.1471 - val_accuracy: 0.8667 - val_loss: 0.2702
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 1.0000 - loss: 0.0729 - val_accuracy: 0.8667 - val_loss: 0.2276
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.8571 - loss: 0.3961
Training MobileNetV2...
Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2512 - loss: 1.6726 - val_accuracy: 0.7333 - val_loss: 0.9253
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.5753 - loss: 1.1851 - val_accuracy: 0.6000 - val_loss: 0.8532
Epoch 3/5
2/2 ━━━━━

In [6]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate models and compute additional metrics
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")

    # Get predictions
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    # Generate classification report
# Generate classification report
    report = classification_report(true_labels, predicted_labels, labels=[0, 1, 2, 3], target_names=label_mapping.keys(), digits=4)

    print(f"\n{model_name} Classification Report:\n{report}")

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    print(f"\n{model_name} Confusion Matrix:\n{cm}\n")

    # Compute additional metrics
    TN = np.diag(cm).sum() - np.diag(cm)  # True Negatives
    FP = cm.sum(axis=0) - np.diag(cm)  # False Positives
    FN = cm.sum(axis=1) - np.diag(cm)  # False Negatives
    TP = np.diag(cm)  # True Positives

    Precision = TP / (TP + FP + 1e-7)  # To avoid division by zero
    Recall = TP / (TP + FN + 1e-7)
    F1_score = 2 * (Precision * Recall) / (Precision + Recall + 1e-7)
    NPV = TN / (TN + FN + 1e-7)
    Specificity = TN / (TN + FP + 1e-7)
    Accuracy = np.sum(TP) / np.sum(cm)

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {Accuracy:.4f}")
    print(f"Precision: {np.mean(Precision):.4f}")
    print(f"Recall: {np.mean(Recall):.4f}")
    print(f"F1-score: {np.mean(F1_score):.4f}")
    print(f"NPV: {np.mean(NPV):.4f}")
    print(f"Specificity: {np.mean(Specificity):.4f}\n")


Evaluating EfficientNetB0...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me


EfficientNetB0 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.7500    1.0000    0.8571         3
Infrastructure Problem     1.0000    0.6667    0.8000         3
          Overcrowding     1.0000    1.0000    1.0000         1
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.8571         7
             macro avg     0.6875    0.6667    0.6643         7
          weighted avg     0.8929    0.8571    0.8531         7


EfficientNetB0 Confusion Matrix:
[[3 0 0]
 [1 2 0]
 [0 0 1]]

EfficientNetB0 Metrics:
Accuracy: 0.8571
Precision: 0.9167
Recall: 0.8889
F1-score: 0.8857
NPV: 0.9333
Specificity: 0.9167

Evaluating MobileNetV2...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

MobileNetV2 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.6000    1.0000    0.7500         3
Infrastructure Problem     0.500

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me


InceptionV3 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.0000    0.0000    0.0000         3
Infrastructure Problem     0.5000    0.3333    0.4000         3
          Overcrowding     0.0000    0.0000    0.0000         1
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.1429         7
             macro avg     0.1250    0.0833    0.1000         7
          weighted avg     0.2143    0.1429    0.1714         7


InceptionV3 Confusion Matrix:
[[0 1 2]
 [2 1 0]
 [1 0 0]]

InceptionV3 Metrics:
Accuracy: 0.1429
Precision: 0.1667
Recall: 0.1111
F1-score: 0.1333
NPV: 0.2500
Specificity: 0.1944

Evaluating NASNetMobile...
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step

NASNetMobile Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.4000    0.6667    0.5000         3
Infrastructure Problem     1.0000    0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [8]:
def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))  # Resize for model input
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)  # Normalize
        images.append(img_array)
    return np.array(images)  # Convert list to numpy array
def predict_images(models, image_paths, label_mapping):
    processed_images = preprocess_images(image_paths)

    for model_name, model in models.items():
        print(f"\nPredictions for {model_name}:")
        predictions = model.predict(processed_images)
        predicted_classes = np.argmax(predictions, axis=1)

        class_names = list(label_mapping.keys())
        predicted_labels = [class_names[i] for i in predicted_classes]

        for i, image_path in enumerate(image_paths):
            print(f"Image {i+1}: {image_path}")
            print(f"Predicted Label: {predicted_labels[i]}")
            print(f"Class Probabilities: {predictions[i]}")
            print("-" * 50)
image_paths = [
    "/content/Screenshot 2025-03-20 055819.png",
    "/content/Screenshot 2025-03-20 060003.png",
    "/content/Screenshot 2025-03-20 060117.png",
    "/content/Screenshot 2025-03-20 060203.png",
    "/content/Screenshot 2025-03-20 060238.png"
]  # Replace with actual image paths

predict_images(models, image_paths, label_mapping)



Predictions for EfficientNetB0:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
Image 1: /content/Screenshot 2025-03-20 055819.png
Predicted Label: Cleanliness Issues
Class Probabilities: [9.9465442e-01 2.4156712e-03 2.1220515e-03 8.0787786e-04]
--------------------------------------------------
Image 2: /content/Screenshot 2025-03-20 060003.png
Predicted Label: Cleanliness Issues
Class Probabilities: [0.46546197 0.39904472 0.0566329  0.07886039]
--------------------------------------------------
Image 3: /content/Screenshot 2025-03-20 060117.png
Predicted Label: Cleanliness Issues
Class Probabilities: [0.6770592  0.1292734  0.05699699 0.13667038]
--------------------------------------------------
Image 4: /content/Screenshot 2025-03-20 060203.png
Predicted Label: Overcrowding
Class Probabilities: [0.01707004 0.0208345  0.9489776  0.01311785]
--------------------------------------------------
Image 5: /content/Screenshot 2025-03-20 060238.png
Predicted Label: Cleanliness Issues
Class Probabili

In [15]:
import tensorflow as tf
import json
import requests
import numpy as np
from tensorflow.keras.applications import (
    EfficientNetB0, MobileNetV2, ResNet50, VGG16, VGG19, DenseNet121, DenseNet201,
    Xception, InceptionV3, NASNetMobile
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from io import BytesIO
from PIL import Image

# Function to download and process images
def load_data(jsonl_path):
    if "valid.jsonl" in jsonl_path:
        raise ValueError("This script is not allowed to process valid.jsonl")

    images, labels = [], []
    with open(jsonl_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            try:
                image_url = data['messages'][2]['content'][0]['image_url']['url']
                label = data['messages'][3]['content']

                # Download image
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content)).convert('RGB')
                img = img.resize((224, 224))  # Resize for model input
                img_array = image.img_to_array(img)
                img_array = preprocess_input(img_array)

                images.append(img_array)
                labels.append(label)
            except KeyError as e:
                print(f"Skipping entry due to missing key: {e}")

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Load train and test data only
train_images, train_labels = load_data("train1.jsonl")
test_images, test_labels = load_data("test1.jsonl")

# Label encoding
label_mapping = {'Cleanliness Issues': 0, 'Infrastructure Problem': 1, 'Overcrowding': 2, 'Suspicious': 3}
train_labels = np.array([label_mapping[label] for label in train_labels])
test_labels = np.array([label_mapping[label] for label in test_labels])

# Convert to categorical
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=4)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=4)

# Function to create model
def create_model(base_model):
    base_model.trainable = False  # Freeze base model
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Define models
models = {
    "EfficientNetB0": create_model(EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "MobileNetV2": create_model(MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "ResNet50": create_model(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "VGG16": create_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "VGG19": create_model(VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "DenseNet121": create_model(DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "DenseNet201": create_model(DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "Xception": create_model(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "InceptionV3": create_model(InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
    "NASNetMobile": create_model(NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
}

# Compile models
for model_name, model in models.items():
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train models
history = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(train_images, train_labels, epochs=5)  # Removed validation data
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    history[model_name] = test_acc

# Print test accuracy for each model
for model_name, acc in history.items():
    print(f"{model_name} Test Accuracy: {acc:.2f}")


Training EfficientNetB0...
Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.2528 - loss: 1.5578
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7906 - loss: 0.8158
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.8905 - loss: 0.4403
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9887 - loss: 0.2348
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 1.0000 - loss: 0.1246
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.9375 - loss: 0.2328
Training MobileNetV2...
Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 979ms/step - accuracy: 0.2867 - loss: 1.5113
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 995ms/step - accuracy: 0.4970 - loss: 1.1038
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 939ms/step - accuracy: 0.7489 - loss: 0.7735
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9218 - loss: 0.5939   
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9340 - loss: 0.4872
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6250 - loss: 0.7

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate models and compute additional metrics
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")

    # Get predictions
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    # Generate classification report
# Generate classification report
    report = classification_report(true_labels, predicted_labels, labels=[0, 1, 2, 3], target_names=label_mapping.keys(), digits=4)

    print(f"\n{model_name} Classification Report:\n{report}")

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    print(f"\n{model_name} Confusion Matrix:\n{cm}\n")

    # Compute additional metrics
    TN = np.diag(cm).sum() - np.diag(cm)  # True Negatives
    FP = cm.sum(axis=0) - np.diag(cm)  # False Positives
    FN = cm.sum(axis=1) - np.diag(cm)  # False Negatives
    TP = np.diag(cm)  # True Positives

    Precision = TP / (TP + FP + 1e-7)  # To avoid division by zero
    Recall = TP / (TP + FN + 1e-7)
    F1_score = 2 * (Precision * Recall) / (Precision + Recall + 1e-7)
    NPV = TN / (TN + FN + 1e-7)
    Specificity = TN / (TN + FP + 1e-7)
    Accuracy = np.sum(TP) / np.sum(cm)

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {Accuracy:.4f}")
    print(f"Precision: {np.mean(Precision):.4f}")
    print(f"Recall: {np.mean(Recall):.4f}")
    print(f"F1-score: {np.mean(F1_score):.4f}")
    print(f"NPV: {np.mean(NPV):.4f}")
    print(f"Specificity: {np.mean(Specificity):.4f}\n")


Evaluating EfficientNetB0...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me


EfficientNetB0 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.8571    1.0000    0.9231         6
Infrastructure Problem     1.0000    0.6667    0.8000         3
          Overcrowding     1.0000    1.0000    1.0000         7
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.9375        16
             macro avg     0.7143    0.6667    0.6808        16
          weighted avg     0.9464    0.9375    0.9337        16


EfficientNetB0 Confusion Matrix:
[[6 0 0]
 [1 2 0]
 [0 0 7]]

EfficientNetB0 Metrics:
Accuracy: 0.9375
Precision: 0.9524
Recall: 0.8889
F1-score: 0.9077
NPV: 0.9762
Specificity: 0.9667

Evaluating MobileNetV2...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

MobileNetV2 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.6000    0.5000    0.5455         6
Infrastructure Problem     0.500

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

ResNet50 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     1.0000    0.8333    0.9091         6
Infrastructure Problem     0.5000    1.0000    0.6667         3
          Overcrowding     1.0000    0.7143    0.8333         7
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.8125        16
             macro avg     0.6250    0.6369    0.6023        16
          weighted avg     0.9062    0.8125    0.8305        16


ResNet50 Confusion Matrix:
[[5 1 0]
 [0 3 0]
 [0 2 5]]

ResNet50 Metrics:
Accuracy: 0.8125
Precision: 0.8333
Recall: 0.8492
F1-score: 0.8030
NPV: 0.8963
Specificity: 0.9231

Evaluating VGG16...


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Xception Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.2500    0.1667    0.2000         6
Infrastructure Problem     0.5000    0.3333    0.4000         3
          Overcrowding     0.6667    0.8571    0.7500         7
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.5000        16
             macro avg     0.3542    0.3393    0.3375        16
          weighted avg     0.4792    0.5000    0.4781        16


Xception Confusion Matrix:
[[1 1 3 1]
 [2 1 0 0]
 [1 0 6 0]
 [0 0 0 0]]

Xception Metrics:
Accuracy: 0.5000
Precision: 0.3542
Recall: 0.3393
F1-score: 0.3375
NPV: 0.7569
Specificity: 0.7160

Evaluating InceptionV3...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

InceptionV3 Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.5000    0.5000    0.5000         6
Infrastructure Problem     0.3333

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step

NASNetMobile Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.5000    0.5000    0.5000         6
Infrastructure Problem     1.0000    0.3333    0.5000         3
          Overcrowding     0.6667    0.8571    0.7500         7
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.6250        16
             macro avg     0.5417    0.4226    0.4375        16
          weighted avg     0.6667    0.6250    0.6094        16


NASNetMobile Confusion Matrix:
[[3 0 3]
 [2 1 0]
 [1 0 6]]

NASNetMobile Metrics:
Accuracy: 0.6250
Precision: 0.7222
Recall: 0.5635
F1-score: 0.5833
NPV: 0.7727
Specificity: 0.7571



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [17]:
def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))  # Resize for model input
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)  # Normalize
        images.append(img_array)
    return np.array(images)  # Convert list to numpy array
def predict_images(models, image_paths, label_mapping):
    processed_images = preprocess_images(image_paths)

    for model_name, model in models.items():
        print(f"\nPredictions for {model_name}:")
        predictions = model.predict(processed_images)
        predicted_classes = np.argmax(predictions, axis=1)

        class_names = list(label_mapping.keys())
        predicted_labels = [class_names[i] for i in predicted_classes]

        for i, image_path in enumerate(image_paths):
            print(f"Image {i+1}: {image_path}")
            print(f"Predicted Label: {predicted_labels[i]}")
            print(f"Class Probabilities: {predictions[i]}")
            print("-" * 50)
image_paths = [
    "/content/Screenshot 2025-03-20 055819.png",
    "/content/Screenshot 2025-03-20 060003.png",
    "/content/Screenshot 2025-03-20 060117.png",
    "/content/Screenshot 2025-03-20 060203.png",
    "/content/Screenshot 2025-03-20 060238.png"
]  # Replace with actual image paths

predict_images(models, image_paths, label_mapping)


Predictions for EfficientNetB0:
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Image 1: /content/Screenshot 2025-03-20 055819.png
Predicted Label: Cleanliness Issues
Class Probabilities: [0.98088616 0.01015207 0.00435738 0.00460436]
--------------------------------------------------
Image 2: /content/Screenshot 2025-03-20 060003.png
Predicted Label: Infrastructure Problem
Class Probabilities: [0.1806316  0.5461529  0.18665023 0.08656529]
--------------------------------------------------
Image 3: /content/Screenshot 2025-03-20 060117.png
Predicted Label: Cleanliness Issues
Class Probabilities: [0.5356177  0.15069614 0.08107778 0.2326084 ]
--------------------------------------------------
Image 4: /content/Screenshot 2025-03-20 060203.png
Predicted Label: Overcrowding
Class Probabilities: [0.00893952 0.00747164 0.9726294  0.01095932]
--------------------------------------------------
Image 5: /content/Screenshot 2025-03-20 060238.png
Predicted Label: Cleanliness Issues
Class Probabilities: [0.47

In [4]:
import numpy as np
import json
import requests
from io import BytesIO
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Function to download and process images
def load_data(jsonl_path):
    if "valid.jsonl" in jsonl_path:
        raise ValueError("This script is not allowed to process valid.jsonl")

    images, labels = [], []
    with open(jsonl_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            try:
                image_url = data['messages'][2]['content'][0]['image_url']['url']
                label = data['messages'][3]['content']

                # Download image
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content)).convert('RGB')
                img = img.resize((224, 224))  # Resize for model input
                img_array = np.array(img).flatten()  # Convert to 1D feature vector

                images.append(img_array)
                labels.append(label)
            except KeyError as e:
                print(f"Skipping entry due to missing key: {e}")

    return np.array(images), np.array(labels)

# Load train and test data only
train_images, train_labels = load_data("train1.jsonl")
test_images, test_labels = load_data("test1.jsonl")

# Label encoding
label_mapping = {'Cleanliness Issues': 0, 'Infrastructure Problem': 1, 'Overcrowding': 2, 'Suspicious': 3}
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Normalize features
scaler = StandardScaler()
train_images = scaler.fit_transform(train_images)
test_images = scaler.transform(test_images)

# Define models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

# Train models
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(train_images, train_labels)

# Evaluate models and compute additional metrics
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")

    # Get predictions
    predicted_labels = model.predict(test_images)
    probabilities = model.predict_proba(test_images) if hasattr(model, 'predict_proba') else None

    # Generate classification report
    report = classification_report(test_labels, predicted_labels, labels=[0, 1, 2, 3], target_names=label_mapping.keys(), digits=4)
    print(f"\n{model_name} Classification Report:\n{report}")

    # Compute confusion matrix
    cm = confusion_matrix(test_labels, predicted_labels)
    print(f"\n{model_name} Confusion Matrix:\n{cm}\n")

    # Compute additional metrics
    TN = np.diag(cm).sum() - np.diag(cm)  # True Negatives
    FP = cm.sum(axis=0) - np.diag(cm)  # False Positives
    FN = cm.sum(axis=1) - np.diag(cm)  # False Negatives
    TP = np.diag(cm)  # True Positives

    Precision = TP / (TP + FP + 1e-7)  # To avoid division by zero
    Recall = TP / (TP + FN + 1e-7)
    F1_score = 2 * (Precision * Recall) / (Precision + Recall + 1e-7)
    NPV = TN / (TN + FN + 1e-7)
    Specificity = TN / (TN + FP + 1e-7)
    Accuracy = np.sum(TP) / np.sum(cm)

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {Accuracy:.4f}")
    print(f"Precision: {np.mean(Precision):.4f}")
    print(f"Recall: {np.mean(Recall):.4f}")
    print(f"F1-score: {np.mean(F1_score):.4f}")
    print(f"NPV: {np.mean(NPV):.4f}")
    print(f"Specificity: {np.mean(Specificity):.4f}\n")

# Function to preprocess new images
def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))  # Resize for model input
        img_array = np.array(img).flatten()  # Convert to 1D feature vector
        images.append(img_array)
    return scaler.transform(np.array(images))  # Normalize

# Function to predict labels for new images
def predict_images(models, image_paths, label_mapping):
    processed_images = preprocess_images(image_paths)

    for model_name, model in models.items():
        print(f"\nPredictions for {model_name}:")
        predicted_classes = model.predict(processed_images)
        class_names = list(label_mapping.keys())
        predicted_labels = [class_names[i] for i in predicted_classes]

        for i, image_path in enumerate(image_paths):
            print(f"Image {i+1}: {image_path}")
            print(f"Predicted Label: {predicted_labels[i]}")
            print("-" * 50)

# Provide image paths for prediction
image_paths = [
    "/content/Screenshot 2025-03-20 055819.png",
    "/content/Screenshot 2025-03-20 060003.png",
    "/content/Screenshot 2025-03-20 060117.png",
    "/content/Screenshot 2025-03-20 060203.png",
    "/content/Screenshot 2025-03-20 060238.png"
]  # Replace with actual image paths

predict_images(models, image_paths, label_mapping)


Training SVM...
Training Random Forest...
Training KNN...
Training Logistic Regression...
Evaluating SVM...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me


SVM Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.4545    0.8333    0.5882         6
Infrastructure Problem     1.0000    0.6667    0.8000         3
          Overcrowding     0.6667    0.2857    0.4000         7
            Suspicious     0.0000    0.0000    0.0000         0

              accuracy                         0.5625        16
             macro avg     0.5303    0.4464    0.4471        16
          weighted avg     0.6496    0.5625    0.5456        16


SVM Confusion Matrix:
[[5 0 1]
 [1 2 0]
 [5 0 2]]

SVM Metrics:
Accuracy: 0.5625
Precision: 0.7071
Recall: 0.5952
F1-score: 0.5961
NPV: 0.7528
Specificity: 0.7583

Evaluating Random Forest...

Random Forest Classification Report:
                        precision    recall  f1-score   support

    Cleanliness Issues     0.5000    0.8333    0.6250         6
Infrastructure Problem     1.0000    0.6667    0.8000         3
          Overcrowding     0.7500